In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

In [7]:
# Load the dataset
train_df = pd.read_csv('csv_files/train.csv')

# Convert binary categorical features to 0 and 1
binary_features = ['CryoSleep', 'VIP']
train_df[binary_features] = train_df[binary_features].astype(bool).astype(int)

# Extract components from 'Cabin' and treat them as separate features
train_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = train_df['Cabin'].str.split('/', expand=True)
train_df['Cabin_Side'] = train_df['Cabin_Side'].map({'P': 1, 'S': 0})

# Explicitly convert 'Cabin_Number' to numeric, handling errors
train_df['Cabin_Number'] = pd.to_numeric(train_df['Cabin_Number'], errors='coerce')

# Drop the original 'Cabin' column as it's no longer needed
train_df.drop('Cabin', axis=1, inplace=True)

# One-hot encode multi-category features
multi_cat_features = ['HomePlanet', 'Destination']
one_hot_encoder = OneHotEncoder(drop='first')  # Adjusted here
encoded_features = one_hot_encoder.fit_transform(train_df[multi_cat_features])
encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)

# Convert to DataFrame and concatenate
encoded_features_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names)

# Drop the original columns and concatenate the new one-hot encoded features
train_df.drop(multi_cat_features, axis=1, inplace=True)
train_df = pd.concat([train_df.reset_index(drop=True), encoded_features_df], axis=1)

# Convert 'Transported' to integer (True=1, False=0) for modeling
train_df['Transported'] = train_df['Transported'].astype(int)

# Define numeric features - ensure this list only contains numeric features that were not one-hot encoded
numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_Number', 'Cabin_Side']
# Imputer for numeric features
imputer = SimpleImputer(strategy='median')
train_df[numeric_features] = imputer.fit_transform(train_df[numeric_features])

# Normalize the numeric features
scaler = StandardScaler()
train_df[numeric_features] = scaler.fit_transform(train_df[numeric_features])

# Prepare features and target for the model
features = [col for col in train_df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck' ]]  # Exclude non-features
X = train_df[features]
y = train_df['Transported']

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Building the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluate on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=1)
print(f'Validation Accuracy: {val_acc}')


Epoch 1/50


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.7179 - loss: 0.5608 - val_accuracy: 0.7798 - val_loss: 0.4580
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.7920 - loss: 0.4405 - val_accuracy: 0.7849 - val_loss: 0.4410
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.7935 - loss: 0.4327 - val_accuracy: 0.7872 - val_loss: 0.4416
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.8018 - loss: 0.4214 - val_accuracy: 0.7792 - val_loss: 0.4376
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.8025 - loss: 0.4199 - val_accuracy: 0.7803 - val_loss: 0.4379
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.8076 - loss: 0.4101 - val_accuracy: 0.7872 - val_loss: 0.4278
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.8058 - loss: 0.4086 - val_accuracy: 0.7913 - val_loss: 0.4315
Epoch 8/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - accuracy: 0.7980 - loss: 0.4175 - val_accurac

In [13]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential([
        Dense(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            activation='tanh',
            input_shape=(X_train.shape[1],)
        ),
        Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.3, step=0.1)),
        Dense(
            units=hp.Int('units_2', min_value=16, max_value=128, step=16),
            activation='relu'
        ),
        Dense(1, activation='relu')
    ])
    
    model.compile(
        optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,  # Number of random configurations to try
    executions_per_trial=2,  # Number of times to train the model per trial
    directory='random_search',
    project_name='titanic_optimization'
)

tuner.search(
    X_train, y_train,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best number of units in the first dense layer: {best_hps.get('units_1')}
Best dropout rate in the first dropout layer: {best_hps.get('dropout_1')}
Best number of units in the second dense layer: {best_hps.get('units_2')}
Best learning rate for the optimizer: {best_hps.get('learning_rate')}
""")

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

# Evaluate on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=1)
print(f'Validation Accuracy with Random Search Tuning: {val_acc}')


Reloading Tuner from random_search/titanic_optimization/tuner0.json

Best number of units in the first dense layer: 224
Best dropout rate in the first dropout layer: 0.2
Best number of units in the second dense layer: 80
Best learning rate for the optimizer: 0.0005668473081877774

Epoch 1/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 721us/step - accuracy: 0.6953 - loss: 2.0088 - val_accuracy: 0.7757 - val_loss: 0.7743
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.7761 - loss: 0.9245 - val_accuracy: 0.6440 - val_loss: 3.2966
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.6642 - loss: 3.0303 - val_accuracy: 0.6647 - val_loss: 3.1645
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step - accuracy: 0.7144 - loss: 2.8579 - val_accuracy: 0.7648 - val_loss: 1.8299
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.7624 - loss: 2.1083 - val_accuracy: 0.7251 - val_loss: 2.0573
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - accuracy: 0.

In [9]:
# Load the test dataset
test_df = pd.read_csv('csv_files/test.csv')

# Convert binary categorical features to 0 and 1 for the test data
test_df[binary_features] = test_df[binary_features].astype(bool).astype(int)

# Extract components from 'Cabin' and treat them as separate features for the test data
test_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = test_df['Cabin'].str.split('/', expand=True)
test_df['Cabin_Side'] = test_df['Cabin_Side'].map({'P': 1, 'S': 0})
test_df['Cabin_Number'] = pd.to_numeric(test_df['Cabin_Number'], errors='coerce')

# Drop the original 'Cabin' column as it's no longer needed for the test data
test_df.drop('Cabin', axis=1, inplace=True)

# One-hot encode multi-category features for the test data using the same encoder
encoded_features_test = one_hot_encoder.transform(test_df[multi_cat_features])
encoded_features_test_df = pd.DataFrame(encoded_features_test.toarray(), columns=encoded_feature_names)

# Drop the original multi-category columns and concatenate the new one-hot encoded features for the test data
test_df.drop(multi_cat_features, axis=1, inplace=True)
test_df = pd.concat([test_df.reset_index(drop=True), encoded_features_test_df.reset_index(drop=True)], axis=1)

# Impute missing values for numeric features only in the test data
test_df[numeric_features] = imputer.transform(test_df[numeric_features])

# Normalize the numeric features in the test data
test_df[numeric_features] = scaler.transform(test_df[numeric_features])

# Prepare the features for prediction
X_test = test_df[features]

# Make predictions with the TensorFlow model
y_pred_test_proba = model.predict(X_test)
y_pred_test = (y_pred_test_proba > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_pred_test
})

# Convert predictions back to boolean (True/False) if necessary
submission_df['Transported'] = submission_df['Transported'].astype(bool)

# Save the submission file
submission_df.to_csv('tensorflow_resultsimple.csv', index=False)


134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step
